<a href="https://colab.research.google.com/github/sgu20191816/bok_choy/blob/main/%EC%B2%AD%EA%B2%BD%EC%B1%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


결측치 및 이상치 처리가 쉬운 애들:
 1. led 관련
 2. 펌프 관련
 3. 기준 온도 - (난방 + 냉방) / 2

결측치 채우기 어려운 애들(변수 간의 상관관계가 얽혀 있어서):
 1. 내외부 온도
 2. 내외부 습도
 3. 내외부 팬 상태
 4. 냉난방 부하






In [ ]:
import random
import pandas as pd
import numpy as np
import os
import warnings
import glob
warnings.filterwarnings(action='ignore')

import xgboost

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from xgboost import plot_importance
%matplotlib inline

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
import statistics # 중앙값

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from collections import Counter
scaler_r = RobustScaler()
scaler_s = StandardScaler()
scaler_m = MinMaxScaler()
from sklearn.preprocessing import QuantileTransformer
quan = QuantileTransformer(output_distribution='normal')

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

#import autosklearn.regression
#from autosklearn.regression import AutoSklearnRegressor

from sklearn.preprocessing import PolynomialFeatures
#from sklearn.neighbors import _dist_metrics

import seaborn as sns


In [ ]:
def IQR(df) :

  q3 = df.quantile(0.75)
  q1 = df.quantile(0.25)

  iqr = q3 - q1
  df = df.where(df < (q3 +1.5 *iqr), (q3 +1.5 *iqr))
  df = df.where(df > (q1 -1.5 *iqr ),(q1 -1.5 *iqr))
  return df

In [ ]:
all_input_list = sorted(glob.glob('/content/gdrive/MyDrive/open/train_input/*.csv'))
all_target_list = sorted(glob.glob('/content/gdrive/MyDrive/open/train_target/*.csv'))

train_input_list = all_input_list
train_target_list = all_target_list

test_input_list = sorted(glob.glob('/content/gdrive/MyDrive/open/test_input/*.csv'))
test_target_list = sorted(glob.glob('/content/gdrive/MyDrive/open/test_target/*.csv'))

In [ ]:
'''for i in train_input_list:
  print(i)
  temp = pd.read_csv(i)
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  temp['년월일'] = pd.to_datetime(temp['년월일'])
  cols = temp.columns
  for j in cols:
    temp[j] = IQR(temp[j])
# 날짜를 기준으로 그룹화
  temp_mean = temp.groupby('년월일').mean()
  x = [ i for i in range(len(temp))]
  plt.figure(figsize = (30,5))
  plt.plot(x, temp['내부온도관측치'])
  plt.plot(x, temp['외부온도관측치'])
  #plt.plot(x, temp_mean['기준온도'])
  plt.show()'''

In [ ]:
sum_ = []
for i in train_target_list:
  df_1 = pd.read_csv(i)
  df_1 = df_1.drop(columns = '시간')
  df_1['rate'] = IQR(df_1['rate'])
  sum = 1
  sum_rate = []
  for j in df_1['rate']:
    sum = sum * (j + 1)
    sum_rate.append(sum)
  sum_.append(sum_rate)
  x = [ i for i in range(len(df_1))]
  plt.plot(x, sum_rate)
  plt.show()
  print(sum)

In [ ]:
# 데이터 처리


df_target = pd.DataFrame() # 데이터프레임
len_target = []

for i in train_target_list :
  #i == 56  or i == 37 or i== 43 or i == 44 or i == 40
  if i == train_target_list[56] or i == train_target_list[37] or i == train_target_list[43] or i == train_target_list[44] or i == train_target_list[40]:
    len_target.append(0)
    continue
  target = pd.read_csv(i) # csv파일 읽고
  target['rate'] = IQR(target['rate'])
  #target['rate'] = med(target['rate']

  len_target.append(len(target)) # 케이스 별 타겟의 개수
  df_target= pd.concat([df_target, target['rate']], ignore_index = False)


In [ ]:
#case 1
temp = pd.read_csv(train_input_list[0])
cols = temp.columns
cnt = -1
for i in cols:
  cnt += 1
  if i == '시간':
    continue
  if '상태' in i:
    temp[i].where(temp[i] == 201, 0)
    temp[cols[cnt + 1]].where(temp[i]!=0, 0)
    continue
  temp[i] = IQR(temp[i])


#시간 칼럼을 통해 년월일 칼럼 생성
temp['시간'] = pd.to_datetime(temp['시간'])
temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
temp['년월일'] = pd.to_datetime(temp['년월일'])

# 날짜를 기준으로 그룹화
temp_mean = temp.groupby('년월일').mean()

# 인덱스를 타겟으로 설정
temp_mean = temp_mean.reset_index(drop=True)
temp_mean['target'] = sum_[0]

# 스케일링

scaler_m.fit(temp_mean)
df = scaler_m.fit_transform(temp_mean)
df = pd.DataFrame(df, columns=temp_mean.columns)

# 결측치 제거


In [ ]:
for i in range(1,len(train_input_list)):
  '''if i == 56 or i == 37 or i== 43 or i == 44 or i == 40:
    print('{} out'.format(i))
    continue'''
  temp = pd.read_csv(train_input_list[i])
  #temp = temp.ffill()
  temp.columns = cols
  cnt = -1
  for j in cols:
    cnt += 1
    if j == '시간':
      continue
    if '상태' in j:
      temp[j].where(temp[j] == 201, 0)
      temp[cols[cnt + 1]].where(temp[j]!=0, 0)
      continue
    temp[j] = IQR(temp[j])
  temp.dropna(subset=['시간'],inplace=True)
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  print(i)
  temp['년월일'] = pd.to_datetime(temp['년월일'])
  temp_mean = temp.groupby('년월일')

  break
  if i == 42:
    temp_mean = temp_mean.drop('2022-03-21', axis = 0)
  temp_mean = temp_mean.reset_index(drop=True)
  temp_mean['target'] = sum_[i]


  scaler_m.fit(temp_mean)

  df_sam = scaler_m.transform(temp_mean)
  df_sam = pd.DataFrame(df_sam, columns=temp_mean.columns)

  df = pd.concat([df, df_sam])

df = df.reset_index(drop=True)


1


In [ ]:
df.isna().sum()

내부온도관측치          0
내부습도관측치          0
CO2관측치           0
EC관측치            0
외부온도관측치          0
외부습도관측치          0
펌프상태             0
펌프작동남은시간         0
최근분무량            0
일간누적분무량          0
냉방상태             0
냉방작동남은시간         0
난방상태             0
난방작동남은시간         0
내부유동팬상태          0
내부유동팬작동남은시간      0
외부환기팬상태          0
외부환기팬작동남은시간      0
화이트 LED상태        0
화이트 LED작동남은시간    0
화이트 LED동작강도      0
레드 LED상태         0
레드 LED작동남은시간     0
레드 LED동작강도       0
블루 LED상태         0
블루 LED작동남은시간     0
블루 LED동작강도       0
카메라상태            0
냉방온도             0
난방온도             0
기준온도             0
난방부하             0
냉방부하             0
총추정광량            0
백색광추정광량          0
적색광추정광량          0
청색광추정광량          0
target           0
dtype: int64

In [ ]:
df[['target']].plot(figsize=(10,5))
plt.title('병합된 데이터의 target')
plt.show()

In [ ]:
df_corr = df.corr()['target']
df_corr = pd.DataFrame(df_corr.sort_values(ascending=False), columns=['target'])
df_corr.style.background_gradient(cmap='viridis')

,target
target,1.000000
화이트 LED상태,0.733328
블루 LED상태,0.722569
내부습도관측치,0.692338
외부온도관측치,0.688587
CO2관측치,0.684275
레드 LED상태,0.683454
외부습도관측치,0.608553
냉방부하,0.587888
냉방상태,0.552803


In [ ]:
df_corr[abs(df_corr['target'])>=0.12].index


Index(['target', '화이트 LED상태', '블루 LED상태', '내부습도관측치', '외부온도관측치', 'CO2관측치',
       '레드 LED상태', '외부습도관측치', '냉방부하', '냉방상태', '내부온도관측치', '냉방작동남은시간',
       '블루 LED동작강도', '냉방온도', '난방온도', '일간누적분무량', '최근분무량', '화이트 LED작동남은시간',
       '레드 LED작동남은시간', '블루 LED작동남은시간', '펌프상태', '외부환기팬상태', '내부유동팬상태',
       '화이트 LED동작강도', '카메라상태', 'EC관측치'],
      dtype='object')

In [ ]:
feature = ['target', '외부환기팬상태', '내부습도관측치', '적색광추정광량', '외부온도관측치', '레드 LED동작강도',
       '화이트 LED동작강도', '백색광추정광량', 'EC관측치']
#feture = ['target', '내부습도관측치', '외부환기팬작동남은시간', '외부환기팬상태', '외부온도관측치', '외부습도관측치',
  #     'EC관측치']
df = df[feature]
df.head()

,target,외부환기팬상태,내부습도관측치,적색광추정광량,외부온도관측치,레드 LED동작강도,화이트 LED동작강도,백색광추정광량,EC관측치
0,0.000000,0.162444,0.041872,0.0,0.211027,0.000000,0.873380,0.0,1.000000
1,0.003302,0.398031,0.126161,0.0,0.000000,0.987779,1.000000,0.0,0.989590
2,0.008255,0.582285,0.128636,0.0,0.066290,0.986323,0.994020,0.0,0.987665
3,0.006604,1.000000,0.000000,0.0,0.354757,0.987066,0.995024,0.0,0.992238
4,0.016269,0.617440,0.329593,0.0,0.667054,0.988308,0.992481,0.0,0.994209


In [ ]:
X = pd.DataFrame(df)
y = df_target
print(X)
print(y)


X_train, X_val, Y_train, Y_val = train_test_split(X,y, test_size= 0.1, random_state = 123)
dtrain = xgboost.DMatrix(X_train, Y_train)
dval = xgboost.DMatrix(X_val,Y_val)

In [ ]:
model = xgboost.XGBRegressor(
     objective = 'reg:squarederror',
     max_depth = 6,
     n_estimators = 10000,
     eta = 0.05,
     learning_rate = 0.001,
     num_boost_around = 10000,
     min_childe_weight = 0.5,
     seed = 2022,
     sub_sample = 0.9,
     colsample_bytree = 0.9
    )

params = {
    'objective' : 'reg:squarederror',
    'seed' : 2022,
    'max_depth' : 6,
    'eta': 0.05,
    'learning_rate' : 0.001,
    'sub_sample' : 0.9,
    'colsample_bytree' : 0.9,
    'min_childe_weight' : 0.5
}
'''
xgb_model = xgboost.train(params = params,num_boost_round = 4000,dtrain=dtrain, early_stopping_rounds = 100, evals = [ (dtrain,'train'),(dval,'eval')])

model.fit(X,y)

preds = model.predict(X_val)
rmse = np.sqrt(mean_squared_error(Y_val, preds))
print("RMSE :%f" %(rmse) )
'''

In [ ]:
from sklearn.model_selection import KFold

l_model = []
k_fold = KFold(n_splits = 5, shuffle=True, random_state = 123)
for train_idx, val_idx in k_fold.split(X) :
  x_t = X.iloc[train_idx]
  y_t = y.iloc[train_idx]
  x_val = X.iloc[val_idx]
  y_val = y.iloc[val_idx]

  l_model.append(model.fit(x_t,y_t, eval_set= [(x_val,y_val)], early_stopping_rounds = 100))

In [ ]:

for case in range(6) :
  preds = []
  temp = pd.read_csv(test_input_list[case])
  #temp = temp.ffill()
  temp.columns = cols


# 시간 칼럼을 통해 년월일 칼럼 생성
  temp['시간'] = pd.to_datetime(temp['시간'])
  temp['년월일'] = temp['시간'].dt.year.astype(str) + '-' + temp['시간'].dt.month.astype(str) + '-' + temp['시간'].dt.day.astype(str)
  temp['년월일'] = pd.to_datetime(temp['년월일'])

# 날짜를 기준으로 그룹화
  temp_mean = temp.groupby('년월일').mean()



  for model in l_model :
    preds_2 = []
    sum = [1 for i in range(len(temp_mean['CO2관측치']))]
# 인덱스를 타겟으로 설정
    temp_mean = temp_mean.reset_index(drop=True)
    temp_mean['target'] = sum
    #print(temp_mean.iloc[[0]])
    df = scaler_m.transform(temp_mean.loc[[0]])

    df = pd.DataFrame(df, columns=temp_mean.columns)
    df = df[feature]

    preds_2.append(model.predict(df)[0])

    for j in range(1, len(temp_mean)):
      sum[j] = (temp_mean['target'][j-1]) * (1+ preds_2[-1])
      temp_mean['target'] = sum
      #print(temp_mean['target'])

      df = scaler_m.transform(temp_mean.loc[[j]])
      df = pd.DataFrame(df, columns=temp_mean.columns)
      df = df[feature]
      preds_2.append(model.predict(df)[0])
    preds.append(preds_2)
# 스케일링
  #df = scaler_m.transform(temp_mean)
  #df = pd.DataFrame(df, columns=temp_mean.columns)

# 결측치 제거
  #df = df.ffill()

  #x_test = pd.DataFrame(df)
  y_test = pd.read_csv(test_target_list[case]) # 정답
  ans = pd.DataFrame(preds)
  ans = ans.mean()
  #x_test = xgboost.DMatrix(x_test)

  #preds = model.predict(x_test)
  #preds = xgb_model.predict(x_test)

  submit_df = pd.read_csv(test_target_list[case])
  if case == 3 :
    print(len(ans))
    print(ans)
  #preds = preds.reshape(-1).tolist()
  submit_df['rate'][0:] = ans[len(ans)-len(submit_df):]
  submit_df.to_csv(test_target_list[case],index = False)




In [ ]:
print(preds)

In [ ]:
import zipfile
os.chdir("/content/gdrive/MyDrive/open/test_target/")
submission = zipfile.ZipFile("../submission.zip", 'w')
for path in test_target_list:
    path = path.split('/')[-1]
    submission.write(path)
submission.close()